<a href="https://colab.research.google.com/github/Nobel712/Machine-Learning/blob/main/FaKeNews_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import  LogisticRegression
from sklearn.metrics import mean_squared_error,accuracy_score
import re

In [ ]:
df=pd.read_csv('/content/Fakenewtrain.csv')

In [ ]:
df.shape

(20800, 5)

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df=df.fillna(' ')

In [ ]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
df['content']=df['author']+' '+df['title']

In [ ]:
df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [ ]:
x=df.drop(columns='label',axis=1)

In [ ]:
y=df['label']

In [ ]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')
stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
porter=PorterStemmer()
n_words= stopwords.words('english') #specify english stop words only
n_words.append("rt") #append rt for stop word dictionary

from nltk.tokenize import word_tokenize 
import string

In [ ]:
def stemming(text):
  text= text.lower()
  
  # removing unwanted digits ,special chracters from the text
  text= ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", text).split()) #tags
  text= ' '.join(re.sub("^@?(\w){1,15}$", " ", text).split())
    
  text= ' '.join(re.sub("(\w+:\/\/\S+)", " ", text).split())   #Links
  text= ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split()) 
  text= ' '.join(re.sub(r'http\S+', '',text).split())
  
  
  text= ' '.join(re.sub(r'www\S+', '',text).split())
  text= ' '.join(re.sub("\s+", " ",text).split()) #Extrem white Space
  text= ' '.join(re.sub("[^-9A-Za-z ]", "" ,text).split()) #digits 
  text= ' '.join(re.sub('-', ' ', text).split()) 
  text= ' '.join(re.sub('_', ' ', text).split()) #underscore 
  
  #Display available PUNCTUATION for examples for c in string.punctuation:
  #print(f"[{c}]")
  
  # removing stopwards and numbers from STRING library
  table= str.maketrans('', '', string.punctuation+string.digits)
  text = text.translate(table)
  
  # Split Sentence as tokens words 
  tokens = word_tokenize(text)
  
  # converting words to their root forms by STEMMING THE WORDS 
  #stemmed1 = [lemmatizer.lemmatize(word) for word in tokens] #Covert words to their actual root
  stemmed2 = [porter.stem(word) for word in tokens] # Covert words to their rootbut not actual
  
  # Delete each stop words from English stop words
  #words = [w for w in stemmed1 if not w in n_words] #n_words contains English stop words
  words = [w for w in stemmed2 if not w in n_words] #n_words contains English stop words

  text  = ' '.join(words)
    
  return text


In [ ]:
df['content'].apply(stemming)

0        darrel lucu hous dem aid didnt even see comey ...
1        daniel j flynn flynn hillari clinton big woman...
2               consortiumnewscom whi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper ti trump poster child whit...
20796    benjamin hoffman nfl playoff schedul matchup o...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [ ]:
X=df['content'].apply(stemming)

In [ ]:
X.shape

(20800,)

In [ ]:
tfdf=TfidfVectorizer()

In [ ]:
X=tfdf.fit_transform(X)

In [ ]:
X

<20800x17270 sparse matrix of type '<class 'numpy.float64'>'
	with 213654 stored elements in Compressed Sparse Row format>

In [ ]:
print(X)

  (0, 15809)	0.27471712693846806
  (0, 2496)	0.3522678204809727
  (0, 7762)	0.23748098757756048
  (0, 8705)	0.27990136174620206
  (0, 2965)	0.23651595008921583
  (0, 13590)	0.24615374050334243
  (0, 5012)	0.22355397608651997
  (0, 4034)	0.2842738617982941
  (0, 274)	0.2587990454936185
  (0, 3813)	0.2592129671238152
  (0, 7077)	0.20958859264680002
  (0, 8988)	0.34838193577523935
  (0, 3613)	0.3448344009813382
  (1, 1898)	0.15522829955318895
  (1, 2233)	0.3827046218354034
  (1, 16939)	0.3006959147850443
  (1, 1494)	0.2939680964184073
  (1, 2816)	0.19101245895017643
  (1, 6886)	0.19073163391923162
  (1, 5566)	0.7142787648434148
  (1, 3580)	0.263718795320779
  (2, 5448)	0.38309036260765145
  (2, 6040)	0.3442597658606408
  (2, 9700)	0.488967591921114
  (2, 15736)	0.41162159721928604
  :	:
  (20797, 1288)	0.3354793663696822
  (20797, 13237)	0.24832576690265726
  (20797, 12459)	0.27271489088818773
  (20797, 15084)	0.3116863228502509
  (20797, 12254)	0.24785557045449144
  (20797, 9600)	0.29550

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.2,random_state=1
                                           )

In [ ]:
logsr=LogisticRegression()

In [ ]:
logsr.fit(xtrain,ytrain)

LogisticRegression()

In [ ]:
logsr.predict(xtest)

array([0, 1, 0, ..., 1, 0, 0])

In [ ]:
logsr.score(xtest,ytest)

0.9725961538461538

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree

In [ ]:
neighbors=KNeighborsClassifier()

In [ ]:
neighbors.fit(xtrain,ytrain)

KNeighborsClassifier()

In [ ]:
neighbors.predict(xtest)

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
neighbors.score(xtest,ytest)

0.5300480769230769

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rdm=RandomForestClassifier()

In [ ]:
rdm.fit(xtrain,ytrain)

RandomForestClassifier()

In [ ]:
rdm.score(xtest,ytest)

0.9911057692307692

In [ ]:
sv=SVC()

In [ ]:
sv.fit(xtrain,ytrain)

SVC()

In [ ]:
sv.score(xtest,ytest)

0.9872596153846154